In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
import statistics

import torch
from torch import nn
from torch.utils.data import DataLoader
from ApDataset import PmAwsDataset
import numpy as np
import copy
import matplotlib.pyplot as plt
import pandas as pd

batch_size = 512
num_input = 48

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


# Define model
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_classes = num_classes

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc_1 = nn.Linear(hidden_size, 256)
        self.fc_2 = nn.Linear(256, 128)
        self.fc = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.2)

        self.relu = nn.ReLU()

    def forward(self, x):
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        output, (hn, cn) = self.lstm(x, (h_0, c_0))

        hn = hn.view(-1, self.hidden_size)
        out = self.relu(hn)
        out = self.dropout(out)
        out = self.fc_1(out)
        out = self.fc_2(out)
        out = self.relu(out)
        out = self.fc(out)
        return out[:, 0]


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    mean_loss = 0
    count_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)  # y.reshape((-1,1)))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        mean_loss += loss.item()
        count_loss += len(X)

    mean_loss /= count_loss
    print(f"mean loss: {mean_loss:>7f}")


def test(dataloader, model, loss_fn, mode='val'):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    count_test = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()  # y.reshape(-1,1)).item()
            count_test += len(X)
    test_loss /= count_test
    test_loss = np.sqrt(test_loss)
    if mode == 'val':
        print(f"Validation MAE: {test_loss:>7f} \n")
    else:
        print(f"Test MAE: {test_loss:>7f} \n")

    return test_loss


for nf in range(1, 18):

    ap_train = PmAwsDataset("train", num_input=num_input, num_file=nf)
    ap_val = PmAwsDataset("val", num_input=num_input, num_file=nf)
    ap_test = PmAwsDataset("test", num_input=num_input, num_file=nf)

    train_dataloader = DataLoader(ap_train, batch_size=batch_size)
    val_dataloader = DataLoader(ap_val, batch_size=batch_size)
    test_dataloader = DataLoader(ap_test, batch_size=batch_size)

    epochs = 1000
    max_tolerance = 100
    input_size = 6
    num_layers = 1
    hidden_size = 512
    num_classes = 1
    y_pred_lstm = []
    path = f'./model.pt'
    tolerance = 0
    best_val_loss = np.finfo(float).max
    if nf == 1:
        model = LSTM().to(device)
    else:
        model = torch.load(path)

    loss_fn = nn.L1Loss(reduction="sum")
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    for t in range(epochs):
        print(f"Epoch {t + 1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        val_loss = test(val_dataloader, model, loss_fn)
        if val_loss > best_val_loss:
            if tolerance > max_tolerance:
                break
            else:
                tolerance += 1
        else:
            best_val_loss = val_loss
    path = f'./model.pt'
    torch.save(model, path)
    print("Done!")

    path = f'./model.pt'
    net = torch.load(path)

    loss_fn = nn.L1Loss(reduction="sum")
    test_loss = test(test_dataloader, net, loss_fn, mode='test')

    X, y_gt_ = ap_test[0:]
    X = torch.Tensor(X).to(device)
    y = net(X)
    y_gt = y_gt_[0]
    y_pred_lstm.append((y).detach().cpu().numpy()[:])
    y_pred_lstm = np.array(y_pred_lstm)
    y_pred_lstm = np.ravel(y_pred_lstm, order='C')
    print(y_pred_lstm)
    print(y_gt_)
    plt.scatter(y_gt_, y_pred_lstm)
    plt.xlabel("True Values")
    plt.ylabel("Predictions")
    plt.show()

    # Correlation coefficient
    correlation_matrix = np.corrcoef(y_gt_, y_pred_lstm)
    correlation_coefficient = correlation_matrix[0, 1]
    print("Correlation coefficient:", correlation_coefficient)
    # px = np.arange(1, 7013)
    # py1 = y_pred_lstm
    # py2 = y_gt_
    # # 하나의 그래프 영역에 두 개의 그래프 그리기
    # fig, ax = plt.subplots(figsize=(25, 7))
    # ax.plot(px, py1, label='y1')
    # ax.plot(px, py2, label='y2')

    # # 그래프 제목, 레이블 등 설정하기
    # ax.set_title('Comparison of y1 and y2')
    # ax.set_xlabel('X axis')
    # ax.set_ylabel('Y axis')
    # ax.legend()

    # # 그래프 보여주기
    # plt.show()

Using cpu device
Epoch 1
-------------------------------
mean loss: 0.059645
Validation MAE: 0.217426 

Epoch 2
-------------------------------


KeyboardInterrupt: ignored

In [ ]:
path = f'./model.pt'
model = torch.load(path)
model.eval()

ap_pred = PmAwsDataset("pred", num_input=num_input)
pred_dataloader = DataLoader(ap_pred, batch_size=batch_size)

pred_list = []

for i in range(len(ap_pred)):
    X, y = ap_pred[i]
    if np.isnan(y):
        with torch.no_grad():
            tensor_input_data = torch.Tensor(X).to(device)
            tensor_input_data = tensor_input_data.unsqueeze(dim=0)
            pred_data = model(tensor_input_data)
            pred_data = pred_data.detach().cpu().numpy()
            pred_list.append(pred_data)
            for j in range(48):
                ap_pred[i + j + 1][0][47 - j][0] = pred_data

    else:
        continue
ans = pd.read_csv('answer_sample.csv')
pred_list = pd.DataFrame(pred_list)
pred_list.to_csv('pm_data.csv')
ans = ans
result = pd.concat([ans, pred_list], axis=1)

result.to_csv('result.csv')


NameError: ignored